In [ ]:
import csv
import os
import matplotlib
import matplotlib.pyplot as plt

px = 1/plt.rcParams['figure.dpi']  
plt.rcParams['text.usetex'] = True

matplotlib.rc('xtick', labelsize=20) 
matplotlib.rc('ytick', labelsize=20) 

def plot(filename, y_label: str, x_label: str, convert_abs: bool = False, clip_vswr_threshold: int = 0, required_frequency: float = 0.0):
    with open(f"./results/{filename}.txt", "r") as file:
        lines = file.readlines()

        x_axis = []
        y_axis = []
        lines = lines[3:]

        # if convert_abs is False, leave the current plot as it is
        if convert_abs == False:
            for line in lines:
                temp = line[:-1]
                x, y = temp.split("\t")
                x, y = float(x), float(y)

                x_axis.append(x)  
                y_axis.append(y)

        # if convert_abs is True, convert the current plot to absolute (percentage) value
        else:
            for line in lines:
                temp = line[:-1]
                x, y = temp.split("\t")
                x, y = float(x), float(y)

                x_axis.append(x)  
                y_axis.append(( pow(10, (y / 10)) * 100 ))

        # clip vswr value to a threshold
        if clip_vswr_threshold != 0:
            temp_1 = [x_axis[y_axis.index(y)] for y in y_axis if y <= clip_vswr_threshold]
            temp_2 = [y for y in y_axis if y <= clip_vswr_threshold]

            x_axis = temp_1[10:]
            y_axis = temp_2[10:]

        # get the required point y co-ordiante to be marked on the plot
        for freq in x_axis:
            
            if freq == required_frequency:
                point = y_axis[x_axis.index(freq)]
                break
            
            if freq > required_frequency:
                j = x_axis.index(freq)

                p1 = (x_axis[j - 0], y_axis[j - 0])
                p2 = (x_axis[j - 1], y_axis[j - 1])

                x1, y1 = p1[0], p1[1]
                x2, y2 = p2[0], p2[1]

                point = y1 + ( (required_frequency - x1) * (y2 - y1) ) / (x2 - x1)

                break

        fig, ax = plt.subplots(figsize=(600*px, 400*px))

        plt.xlabel(x_label, fontsize=20)
        plt.ylabel(y_label, fontsize=20)

        ax.plot(x_axis, y_axis, color = "crimson")
        ax.plot(required_frequency, point, marker='v', ms = 10, mfc = "black", mec = "black", label = 'points')

        ax.grid()

        fig.savefig(f'./plots/{filename}.png', format='png', dpi=300, bbox_inches = "tight")

In [ ]:
required_frequency = 0.40006

plot(
    filename = "s11",
    y_label = r"$S_{11}$ (dB)",
    x_label = "Frequency (THz)",
    required_frequency = required_frequency)
    
plot(
    filename = "gain", 
    y_label = r"Gain (dBi)", 
    x_label = "Frequency (THz)",
    required_frequency = required_frequency)

plot(
    filename = "directivity",
    y_label = r"Directivity (dBi)",
    x_label = "Frequency (THz)",
    required_frequency = required_frequency)

plot(
    filename = "vswr",
    y_label = r"VSWR", 
    x_label = "Frequency (THz)", 
    clip_vswr_threshold = 10,
    required_frequency = required_frequency)

plot(
    filename = "total efficiency",
    y_label = "Total Efficiency \%",
    x_label = "Frequency (THz)",
    convert_abs = True,
    required_frequency = required_frequency)

plot(
    filename = "radiated efficiency", 
    y_label = "Radiation Efficiency \%",
    x_label = "Frequency (THz)",
    convert_abs = True,
    required_frequency = required_frequency)